In [ ]:
ROOT_DIR = "/content/drive/MyDrive/Mask_RCNN/"

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
# Just in case the pip mrcnn package is installed, uninstall it.
# We want to use the M R-CNN version in our drive.
!pip uninstall mrcnn

# Uninstall the default packages Colab uses so we can use the versions specified in requirements.txt.

!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y

Found existing installation: keras 2.7.0
Uninstalling keras-2.7.0:
  Successfully uninstalled keras-2.7.0
Found existing installation: Keras-Preprocessing 1.1.2
Uninstalling Keras-Preprocessing-1.1.2:
  Successfully uninstalled Keras-Preprocessing-1.1.2
Found existing installation: keras-vis 0.4.1
Uninstalling keras-vis-0.4.1:
  Successfully uninstalled keras-vis-0.4.1
Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Successfully uninstalled h5py-3.1.0


In [ ]:
!pip install -r /content/drive/MyDrive/Mask_RCNN/requirements.txt

     |████████████████████████████████| 377.1 MB 9.2 kB/s 
     |████████████████████████████████| 211 kB 32.3 MB/s 
     |████████████████████████████████| 276 kB 30.0 MB/s 
     |████████████████████████████████| 2.9 MB 28.8 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 3.1 MB 55.0 MB/s 
     |████████████████████████████████| 488 kB 68.0 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 154 kB 68.7 MB/s 
     |████████████████████████████████| 397 kB 69.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0


In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

sys.path.append(os.path.join(ROOT_DIR, "samples/balloon/"))  # To find local version
import balloon

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
# Load datasets
DATASET_PATH = os.path.join(ROOT_DIR, "datasets/balloon")

# Training dataset.
dataset_train = balloon.BalloonDataset()
dataset_train.load_balloon(DATASET_PATH, "train")
dataset_train.prepare()

# Validation dataset
dataset_val = balloon.BalloonDataset()
dataset_val.load_balloon(DATASET_PATH, "val")
dataset_val.prepare()

In [ ]:
# Set up model to prune
import warnings
warnings.filterwarnings('ignore')

BALLOON_DIR = os.path.join(ROOT_DIR, "datasets/balloon")
config = balloon.BalloonConfig()
class InferenceConfig(config.__class__):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

weights_path = '/content/drive/MyDrive/Mask_RCNN/logs/balloon_all_losses/mask_rcnn_balloon_0029.h5'
print("Loading weights from ", weights_path)
model.load_weights(weights_path, by_name=True)






Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights from  /content/drive/MyDrive/Mask_RCNN/logs/balloon_all_losses/mask_rcnn_balloon_0029.h5


In [ ]:
warnings.filterwarnings('ignore')

# Prune layers that contain the following terms
layer_regex_list = {
            # all layers but the backbone
            "heads": r"(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
            # From a specific Resnet stage and up
            "3+": r"(res3.*)|(bn3.*)|(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
            "4+": r"(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
            "5+": r"(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)",
            # All layers
            "all": ".*",
        }

def prune(model, threshold=0.1, layers = 'all'):
    masks = dict()
    layer_regex = layer_regex_list[layers]
    for layer in model.keras_model.layers: 
        if not layer.trainable:
          continue
        if not bool(re.fullmatch(layer_regex, layer.name)):
          continue
        if not layer.__class__.__name__ =='Conv2D':
          continue

        layer_mask = np.ones(layer.filters)
        layer_weights = layer.get_weights()
        l1norms = []
        for i in range(len(layer_mask)):
              filter_weights = np.abs(layer_weights[0][:,:,:,i]) 
              l1norms.append(filter_weights.sum())
        
        num_prune = round(len(layer_mask) * threshold)
        idx = np.argpartition(l1norms, num_prune)
        for i in range(num_prune):
            layer_weights[0][:,:,:,idx[i]] *= 0
            layer_mask[idx[i]] = 0

        layer.set_weights(layer_weights)
        masks[layer.name] = layer_mask
    return masks

# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []

masks = prune(model, threshold=0.3) #TODO: CHANGE THRESHOLD HERE

for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)

np.array(APs).mean()

1/1 [==============================] - 4s
Prediction step: 4.8277974128723145 seconds
1/1 [==============================] - 0s
Prediction step: 0.1677992343902588 seconds
1/1 [==============================] - 0s
Prediction step: 0.17361927032470703 seconds
1/1 [==============================] - 0s
Prediction step: 0.16922783851623535 seconds
1/1 [==============================] - 0s
Prediction step: 0.16980504989624023 seconds
1/1 [==============================] - 0s
Prediction step: 0.16962385177612305 seconds
1/1 [==============================] - 0s
Prediction step: 0.17129230499267578 seconds
1/1 [==============================] - 0s
Prediction step: 0.16649699211120605 seconds
1/1 [==============================] - 0s
Prediction step: 0.17064929008483887 seconds
1/1 [==============================] - 0s
Prediction step: 0.17115497589111328 seconds


0.20000000149011612

In [ ]:
import time

# Fine tune the pruned model
warnings.filterwarnings('ignore')

def get_model_with_pruned_weights(model_ft, masks):
    for layer in model_ft.keras_model.layers: 
      if layer.name in masks:
          layer_mask = masks[layer.name]
          layer_weights = layer.get_weights()   
          for i in range(len(layer_mask)):
              layer_weights[0][:,:,:,i] *= layer_mask[i]
          layer.set_weights(layer_weights)
          layer.trainable = False

from mrcnn.config import Config

class BalloonConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "balloon"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 50

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

config = BalloonConfig()

model_ft = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)
log_path = '/content/drive/MyDrive/Mask_RCNN/logs/balloon_ft_30pc/' # CHANGE DIRECTORY TO SAVE TRAINED MODEL HERE
print("Loading weights ", weights_path)
model_ft.load_weights(weights_path, by_name=True)
model_ft.set_log_dir(log_path)
get_model_with_pruned_weights(model_ft, masks)


model_ft.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=3, 
            layers="all", )

Loading weights  /content/drive/MyDrive/Mask_RCNN/logs/balloon_all_losses/mask_rcnn_balloon_0029.h5

Starting at epoch 0. LR=0.0001

Checkpoint Path: /content/drive/MyDrive/Mask_RCNN/logs/balloon20211130T1852/mask_rcnn_balloon_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchN

In [ ]:
# Check performance after pruning
model_check = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=MODEL_DIR)
#original balloon
#model_path = '/content/drive/MyDrive/Mask_RCNN/logs/balloon_all_losses/mask_rcnn_balloon_0029.h5'
model_path = '/content/drive/MyDrive/Mask_RCNN/logs/balloon_ft_20pc/mask_rcnn_balloon_0002.h5' # CHANGE MODEL PATH HERE
print("Loading weights from ", model_path)

model_check.load_weights(model_path, by_name=True)

APs = []
class_accs = []
class_preds = []
count = 0
res = {"rois":[], "class_ids": [], "scores": [], "masks":[]}
for image_id in dataset_val.image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model_check.detect([image], verbose=0)
    #results = model.detect([image], verbose=0)
    
    r = results[0]
    
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)

    class_acc = utils.compute_mean_class_acc(gt_bbox, gt_class_id, gt_mask,
                      r["rois"], r["class_ids"], r["scores"], r['masks'])
    class_accs.append(class_acc)

    class_pred = compute_mean_class_prob(gt_bbox, gt_class_id, gt_mask,
                      r["rois"], r["class_ids"], r["scores"], r['masks'])
    class_preds.append(class_pred)
    count+=1
    
print("AP", np.array(APs).mean())
print("Accuracy", np.array(class_accs).mean())
print("class pred", np.array(class_preds).mean())
print("count", count)

Loading weights from  /content/drive/MyDrive/Mask_RCNN/logs/balloon_ft_20pc/mask_rcnn_balloon_0002.h5
1/1 [==============================] - 3s
Prediction step: 3.421720266342163 seconds
1/1 [==============================] - 0s
Prediction step: 0.18321990966796875 seconds
1/1 [==============================] - 0s
Prediction step: 0.17503070831298828 seconds
1/1 [==============================] - 0s
Prediction step: 0.17762541770935059 seconds
1/1 [==============================] - 0s
Prediction step: 0.19106626510620117 seconds
1/1 [==============================] - 0s
Prediction step: 0.1845874786376953 seconds
1/1 [==============================] - 0s
Prediction step: 0.1762409210205078 seconds
1/1 [==============================] - 0s
Prediction step: 0.17593050003051758 seconds
1/1 [==============================] - 0s
Prediction step: 0.1753368377685547 seconds
1/1 [==============================] - 0s
Prediction step: 0.17839908599853516 seconds
1/1 [============================

In [ ]:
class_preds

[0.9991059899330139,
 0.997659044606345,
 0.9864219890700446,
 0.9792168041070303,
 0.992611899971962,
 0.9853130153247288,
 0.9996287822723389,
 0.998497724533081,
 0.9991932511329651,
 0.9825622042020162,
 0.9998321533203125,
 0.9964537620544434,
 0.9997583429018656]

In [ ]:
def calculate_mAP(model, iou=0.5, range=False, bbox=False):
  APs = []

  for image_id in dataset_val.image_ids:
      # Load image and ground truth data
      image, image_meta, gt_class_id, gt_bbox, gt_mask =\
          modellib.load_image_gt(dataset_val, inference_config,
                                image_id, use_mini_mask=False)
      molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
      # Run object detection
      results = model.detect([image], verbose=0)
      r = results[0]
      # Compute AP
      if range:
        if bbox:
          AP =\
              utils.compute_ap_range_boxes(gt_bbox, gt_class_id, gt_mask,
                              r["rois"], r["class_ids"], r["scores"], r['masks'], verbose=0)
        else:
          AP =\
              utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask,
                              r["rois"], r["class_ids"], r["scores"], r['masks'], verbose=0)
      else:
        if bbox:
          AP, precisions, recalls, overlaps =\
              utils.compute_ap_boxes(gt_bbox, gt_class_id, gt_mask,
                              r["rois"], r["class_ids"], r["scores"], r['masks'], iou_threshold=iou)
        else:
          AP, precisions, recalls, overlaps =\
              utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                              r["rois"], r["class_ids"], r["scores"], r['masks'], iou_threshold=iou)
      APs.append(AP)

  return np.array(APs).mean()

# Segmentation 
def mAP_table(model):
  seg_iou_50_95 = calculate_mAP(model, range=True)
  seg_iou_50 = calculate_mAP(model, iou=0.5)
  seg_iou_75 = calculate_mAP(model, iou=0.75)

  bbox_iou_50_95 = calculate_mAP(model, range=True, bbox=True)
  bbox_iou_50 = calculate_mAP(model, iou=0.5, bbox=True)
  bbox_iou_75 = calculate_mAP(model, iou=0.75, bbox=True)

  print("Segmentation")
  print("Average Precision (AP) @[ IoU=0.50:0:95 | area= all ] = " + str(seg_iou_50_95))
  print("Average Precision (AP) @[ IoU=0.50      | area= all ] = " + str(seg_iou_50))
  print("Average Precision (AP) @[ IoU=0.75      | area= all ] = " + str(seg_iou_75))
  print("Bounding box")
  print("Average Precision (AP) @[ IoU=0.50:0:95 | area= all ] = " + str(bbox_iou_50_95))
  print("Average Precision (AP) @[ IoU=0.50      | area= all ] = " + str(bbox_iou_50))
  print("Average Precision (AP) @[ IoU=0.75      | area= all ] = " + str(bbox_iou_75))

  return {"50": [bbox_iou_50, seg_iou_50], "75": [bbox_iou_75, seg_iou_75]}

In [ ]:
output_dict = mAP_table(model_check)
print(output_dict)

1/1 [==============================] - 0s
Prediction step: 0.19005584716796875 seconds
1/1 [==============================] - 0s
Prediction step: 0.17976927757263184 seconds
1/1 [==============================] - 0s
Prediction step: 0.17612552642822266 seconds
1/1 [==============================] - 0s
Prediction step: 0.176499605178833 seconds
1/1 [==============================] - 0s
Prediction step: 0.17662549018859863 seconds
1/1 [==============================] - 0s
Prediction step: 0.17805719375610352 seconds
1/1 [==============================] - 0s
Prediction step: 0.17649054527282715 seconds
1/1 [==============================] - 0s
Prediction step: 0.1760239601135254 seconds
1/1 [==============================] - 0s
Prediction step: 0.17598223686218262 seconds
1/1 [==============================] - 0s
Prediction step: 0.18107342720031738 seconds
1/1 [==============================] - 0s
Prediction step: 0.17900323867797852 seconds
1/1 [==============================] - 0s
Pred